# 线程替代方案
- subprocess
    - 完全跳过线程,使用进程
    - 是派生进程的主要替代方案
- multiprocessing
    - 使用threading接口派生,使用子进程
    - 允许为多核或者多cpu派生进程,接口跟threading非常相似
- concurrent.futures
    - 新的异步执行模块
    - 任务级别的操作
    
# 多进程
- 进程间通讯(InterprocessCommunication, IPC)
- 进程之间无任何共享状态
- 进程的创建
    - 直接生成Prcess实例对象, 案例01.py

In [ ]:
import multiprocessing
from time import sleep, ctime

# interval 是传入的参数 睡眠多少时间
def clock(interval):
    while True:
        print("The time is %s" % ctime())
        sleep(interval)
        
if __name__ == "__main__":
    p = multiprocessing.Process(target = clock, args=(3,))
    p.start()
    
    while True:
        print("*********")
        sleep(1)

- 派生类, 案例02.py

In [ ]:
import multiprocessing
from time import sleep, ctime

class ClockPrcess(multiprocessing.Process):
    # 两个重要的函数 构造函数和run函数
    
    def __init__(self, interval):
        super().__init__()
        self.interval = interval
        
    def run(self):
        while True:
            print("The time is %s" % ctime())
            sleep(self.interval)
            
if __name__ == "__main__":
    p = ClockPrcess(3)
    p.start()
    
    while True:
        print("**********")
        sleep(1)

- 在os中查看pid, ppid以及他们的关系,案例03.py

In [6]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print("module name:", __name__)
    # 得到父亲进程的id
    print("parent process:", os.getppid())
    # 得到本身的进程id
    print("process id:", os.getpid())
    
def f(name):
    info("function f")
    print("hello", name)
    
if __name__ == "__main__":
    info ("main line")
    p = Process(target = f, args=("bob",))
    p.start()
    p.join()
# 代码结果可以知道 在函数执行的时候 哪个是父类进程 哪个是本身进程

- 生产者消费者模型
    - JoinableQueue,参看案例04.py

In [14]:
import multiprocessing
from time import ctime

# 消费者
def consumer(input_q):
    print("Into consumer:", ctime())
    while True:
        item = input_q.get() # 消费一个(取出一个)
        print("pull", item, "out of q")
        input_q.task_done() # 发出完成任务的信号
        
    #此句未执行，因为q.join()收集到四个task_done()信号后，主进程启动，未等到print此句完成，程序就结束    
    print("Out of consumer:", ctime())
    了
# 生产者
def producer(sequence, output_q):
    print("Into procuder:", ctime())
    # 以下循环,是把sequence一个个拿出来
    for item in sequence:
        # 把sequence里的东西一次次放入output_q里
        output_q.put(item)
        print("put", item, "into q")
    # 提示生产完了(放入完了)
    print("Out of producer:", ctime())
    
# 建立进程
if __name__ == "__main__":
    q = multiprocessing.JoinableQueue()
    # 运行消费者
    cons_p = multiprocessing.Process(target = consumer, args = (q,))
    cons_p.daemon = True
    cons_p.start()
    
    # 生产多个项, sequence代表要发送给消费者的项序列
    # 在实践中, 这可能是生成器的输出或通过一些其他方式生产出来
    sequence = [1,2,3,4]
    producer(sequence, q)
    
    cons_p.join()

- 队列中哨兵的使用,案例05.py

In [ ]:
import multiprocessing
from time import ctime

def consumer(input_q):
    print("Into consumer:", ctime())
    # 区别
    while True:
        item = input_q.get()
        if item is None:
            break
        print("pull", item, "out of q")
    print("Out of consumer:", ctime())
    
def producer(sequence, output_q):
    print("Into producer:", ctime())
    for item in sequence:
        output_q.put(item)
        print("put", item, "into q")
    print("Out of producer:", ctime())
    
if __name__ == "__main__":
    q = multiprocessing.Queue()
    
    con_p = multiprocessing.Process(target = consumer, args=(q,))
    con_p.start()
    
    sequence = [1,2,3,4]
    producer(sequence, q)
    
    q.put(None)
    con_p.join()